In [3]:
versioninfo()

Julia Version 1.0.0
Commit 5d4eaca0c9 (2018-08-08 20:58 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Pentium(R) CPU B980 @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, sandybridge)


## Common packages

In [4]:
using Printf
using LinearAlgebra
using Random
using FFTW

## External libraries

In [6]:
include("../extlibs/extlibs.jl");

## Constants

In [7]:
const Ry2eV = 13.605693009;
const ANG2BOHR = 1.8897261254578281;

## Type aliases

In [8]:
const BlochWavefunc = Array{Array{ComplexF64,2},1};
const Wavefunc = Array{ComplexF64,2};

## Atoms, lattice, etc

In [9]:
include("../src/Atoms.jl");

include("../src/gen_lattice_pwscf.jl");

include("../src/spglib.jl");

include("../src/KPoints.jl");

include("../src/XSF_utils.jl");

include("../src/Utilities.jl");

include("../src/ortho_gram_schmidt.jl");
include("../src/ortho_sqrt.jl");
include("../src/ortho_check.jl");

include("../src/Ylm_real.jl");
include("../src/Ylm_complex.jl");

include("../src/fft_support.jl");

In [10]:
?Atoms

search: Atoms get_Zatoms init_atoms_xyz init_atoms_xyz_ext init_atoms_xyz_string



The type for representing an atomic-scale structures such as molecules and solids.

---

Creates an instance of `Atoms`.

If no arguments are given, then a dummy `Atoms` is returned.

Use only one of `xyz_file`, `xyz_string` and `xyz_string_frac` for specifying number of atoms, atomic symbols, and positions, following xyz format, for example:

```
3

C   0.0   0.0   0.0
O   0.0   0.0   1.5
```

The argument `xyz_file` is used when the structure is given in an xyz file, which in this case `xyz_file` is a string of path to the xyz file. By default the unit used is angstrom. If the coordinate is given in bohr, set `in_bohr=true`.

Alternatively, one can pass the content of xyz file as string in `xyz_string`.

`xyz_string_frac` is the same as `xyz_string`, however the actual coordinates will be transformed by multiplying it with `LatVecs`. This is useful for crystals.
